In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [2]:
import keras_tuner as kt
from keras_tuner import RandomSearch
# from GEH_LOSS import GEH_loss
import geh as ga

In [3]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [4]:
import relative_accuracy as ra

In [5]:
# Load all Inputs and Output Data

# 5 mins (1 step ahead)
Deep_train_5   = np.load("Deep_train_5_multi.npz")['x'] 
Output_train_5 = np.load("Deep_train_5_multi.npz")['y']

Deep_test_5   = np.load("Deep_test_5_multi.npz")['x'] 
Output_test_5 = np.load("Deep_test_5_multi.npz")['y'] 

# 15 mins (3 steps ahead)
Deep_train_15   = np.load("Deep_train_15_multi.npz")['x'] 
Output_train_15 = np.load("Deep_train_15_multi.npz")['y']

Deep_test_15   = np.load("Deep_test_15_multi.npz")['x'] 
Output_test_15 = np.load("Deep_test_15_multi.npz")['y']

# 30 mins (6 steps ahead)
Deep_train_30   = np.load("Deep_train_30_multi.npz")['x'] 
Output_train_30 = np.load("Deep_train_30_multi.npz")['y']

Deep_test_30   = np.load("Deep_test_30_multi.npz")['x'] 
Output_test_30 = np.load("Deep_test_30_multi.npz")['y']

# 60 mins (12 steps ahead)
Deep_train_60   = np.load("Deep_train_60_multi.npz")['x'] 
Output_train_60 = np.load("Deep_train_60_multi.npz")['y']

Deep_test_60   = np.load("Deep_test_60_multi.npz")['x'] 
Output_test_60 = np.load("Deep_test_60_multi.npz")['y']

In [6]:
# Wide Data
Xtrain = joblib.load("002weeks_train_Multi.save") 
Xtest = joblib.load("002weeks_test_Multi.save") 

In [7]:
# 5 mins
# Delete first 15 samples
Wide_train_5 = np.delete(Xtrain, np.s_[0:15], 0)
Wide_test_5 = np.delete(Xtest, np.s_[0:15], 0)

In [8]:
# 15 mins
# Delete first 17 samples
Wide_train_15 = np.delete(Xtrain, np.s_[0:17], 0)
Wide_test_15 = np.delete(Xtest, np.s_[0:17], 0)

In [9]:
# 30 mins
# Delete first 20 samples
Wide_train_30 = np.delete(Xtrain, np.s_[0:20], 0)
Wide_test_30 = np.delete(Xtest, np.s_[0:20], 0)

In [10]:
# 60 mins
# Delete first 27 samples
Wide_train_60 = np.delete(Xtrain, np.s_[0:26], 0)
Wide_test_60 = np.delete(Xtest, np.s_[0:26], 0)

In [11]:
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename) 

In [12]:
Test      = pd.read_csv('01test_scaled.csv', index_col=None, parse_dates=[0])

In [13]:
from pyts.image import GramianAngularField

# Reshape Samples

The first step is to split the input sequences into subsequences that can be processed by the CNN model. Here, each spatio-temporal sample can be split into three sub-samples, each with five time steps. The CNN can interpret each subsequence of five time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [14]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 6                # No of loop detectors
n_seq = 3                     # Subsequences
n_steps = 5                   # time-step per subsequence
# val_percent = 0.07567       # 2 weeks
val_percent = 0.2        
batch_size = 32
n_GAF = 15

n_seq_ = 3                     # Subsequences
n_steps_ = 3                   # time-step per subsequence

In [15]:
# Summation
gasf = GramianAngularField(image_size=1., method='summation', sample_range=None)
# Difference
gadf = GramianAngularField(image_size=1., method='difference', sample_range=None)

#### 5 mins

In [16]:
# Train
Xt5 = np.empty((len(Deep_train_5), n_features, n_GAF, n_GAF))
yt5 = scaler.inverse_transform(Output_train_5)

In [17]:
# Test
Xv5 = np.empty((len(Deep_test_5), n_features, n_GAF, n_GAF))
yv5 = scaler.inverse_transform(Output_test_5)

In [18]:
Deep_train_5 = np.where(np.isclose(Deep_train_5, 1, rtol=0, atol=1e-9), 1, Deep_train_5)
Deep_test_5 = np.where(np.isclose(Deep_test_5, 1, rtol=0, atol=1e-9), 1, Deep_test_5) 

#### 15 mins

In [19]:
# Train
Xt15 = np.empty((len(Deep_train_15), n_features, n_GAF, n_GAF))
yt15 = scaler.inverse_transform(Output_train_15)

In [20]:
# Test
Xv15 = np.empty((len(Deep_test_15), n_features, n_GAF, n_GAF))
yv15 = scaler.inverse_transform(Output_test_15)

In [21]:
Deep_train_15 = np.where(np.isclose(Deep_train_15, 1, rtol=0, atol=1e-9), 1, Deep_train_15)
Deep_test_15 = np.where(np.isclose(Deep_test_15, 1, rtol=0, atol=1e-9), 1, Deep_test_15) 

#### 30 mins

In [22]:
# Train
Xt30 = np.empty((len(Deep_train_30), n_features, n_GAF, n_GAF))
yt30 = scaler.inverse_transform(Output_train_30)

In [23]:
# Test
Xv30 = np.empty((len(Deep_test_30), n_features, n_GAF, n_GAF))
yv30 = scaler.inverse_transform(Output_test_30)

In [24]:
Deep_train_30 = np.where(np.isclose(Deep_train_30, 1, rtol=0, atol=1e-9), 1, Deep_train_30)
Deep_test_30 = np.where(np.isclose(Deep_test_30, 1, rtol=0, atol=1e-9), 1, Deep_test_30)    

#### 60 mins

In [25]:
# Train
Xt60 = np.empty((len(Deep_train_60), n_features, n_GAF, n_GAF))
yt60 = scaler.inverse_transform(Output_train_60)

In [26]:
# Test
Xv60 = np.empty((len(Deep_test_60), n_features, n_GAF, n_GAF))
yv60 = scaler.inverse_transform(Output_test_60)

In [27]:
Deep_train_60 = np.where(np.isclose(Deep_train_60, 1, rtol=0, atol=1e-9), 1, Deep_train_60)
Deep_test_60 = np.where(np.isclose(Deep_test_60, 1, rtol=0, atol=1e-9), 1, Deep_test_60) 

In [28]:
## Check Size

In [29]:
Xt5.shape

(49233, 6, 15, 15)

## Tranformations

### 5 mins

In [30]:
for i in range(len(Xt5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_5[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt5[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [31]:
for i in range(len(Xv5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_5[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv5[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [32]:
Xt5.shape

(49233, 6, 15, 15)

In [33]:
Xv5.shape

(4017, 6, 15, 15)

### 15 mins

In [34]:
for i in range(len(Xt15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_15[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt15[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [35]:
for i in range(len(Xv15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_15[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv15[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [36]:
Xt15.shape

(49231, 6, 15, 15)

In [37]:
Xv15.shape

(4015, 6, 15, 15)

### 30 mins

In [38]:
for i in range(len(Xt30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_30[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt30[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [39]:
for i in range(len(Xv30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_30[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv30[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [40]:
Xt30.shape

(49228, 6, 15, 15)

In [41]:
Xv30.shape

(4012, 6, 15, 15)

### 60 mins

In [42]:
for i in range(len(Xt60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_60[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt30[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [43]:
for i in range(len(Xv60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_60[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv60[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [44]:
Xt60.shape

(49222, 6, 15, 15)

In [45]:
Xv60.shape

(4006, 6, 15, 15)

### Reshaping the Data for Conv2D layer

For Conv2D, there is a need to add one more dimension to show we're dealing with 1 channel (since technically the images are in black and white, only showing values from 0-max flow on a single channel).

Conv1D - strides in 1 dimension
Conv2D - strides in 2 dimensions

In [46]:
# Train data
Xt5  =  np.transpose(Xt5, (0, 2, 3, 1))
Xt15 =  np.transpose(Xt15, (0, 2, 3, 1))
Xt30 =  np.transpose(Xt30, (0, 2, 3, 1))
Xt60 =  np.transpose(Xt60, (0, 2, 3, 1))

# Validation data
Xv5  =  np.transpose(Xv5, (0, 2, 3, 1))
Xv15 =  np.transpose(Xv15, (0, 2, 3, 1))
Xv30 =  np.transpose(Xv30, (0, 2, 3, 1))
Xv60 =  np.transpose(Xv60, (0, 2, 3, 1))

In [47]:
Xt5.shape

(49233, 15, 15, 6)

In [48]:
Xv5.shape

(4017, 15, 15, 6)

In [49]:
# Train data
Xt5  =  Xt5.reshape(Xt5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt15 =  Xt15.reshape(Xt15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt30 =  Xt30.reshape(Xt30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt60 =  Xt60.reshape(Xt60.shape[0], n_seq, n_steps, n_GAF, n_features)

# Validation data
Xv5  =  Xv5.reshape(Xv5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv15 =  Xv15.reshape(Xv15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv30 =  Xv30.reshape(Xv30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv60 =  Xv60.reshape(Xv60.shape[0], n_seq, n_steps, n_GAF, n_features)

In [50]:
Wt5  = Wide_train_5
Wt15 = Wide_train_15
Wt30 = Wide_train_30
Wt60 = Wide_train_60

Wv5  = Wide_test_5
Wv15 = Wide_test_15
Wv30 = Wide_test_30
Wv60 = Wide_test_60

In [51]:
# Train data
Wt5  =  Wt5.reshape(Wt5.shape[0], n_seq_, n_steps_, n_features)
Wt15 =  Wt15.reshape(Wt15.shape[0], n_seq_, n_steps_, n_features)
Wt30 =  Wt30.reshape(Wt30.shape[0], n_seq_, n_steps_, n_features)
Wt60 =  Wt60.reshape(Wt60.shape[0], n_seq_, n_steps_, n_features)

# Validation data
Wv5  =  Wv5.reshape(Wv5.shape[0], n_seq_, n_steps_, n_features)
Wv15 =  Wv15.reshape(Wv15.shape[0], n_seq_, n_steps_, n_features)
Wv30 =  Wv30.reshape(Wv30.shape[0], n_seq_, n_steps_, n_features)
Wv60 =  Wv60.reshape(Wv60.shape[0], n_seq_, n_steps_, n_features)

In [52]:
Xt5.shape

(49233, 3, 5, 15, 6)

In [53]:
Wt5.shape

(49233, 3, 3, 6)

# CNN-LSTM model

In [54]:
from tensorflow import keras
from keras_self_attention import SeqSelfAttention

# Early Stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights = True)

In [55]:
Wide_train_5.shape

(49233, 9, 6)

# 5 mins ahead

In [56]:
def build_model_5(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt5.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt5.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_5w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_5 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_5w)
    
    
    # Deep Model
    # CNN 
    CNN_5 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_5 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_5)
    # LSTM          
    LSTM_5 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_5)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_5, LSTM_5], axis=1)

    # Attention
    Att_5 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_5 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_5.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                                   
    return model_5

In [57]:
tuner5 = RandomSearch(
    build_model_5,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10b-5",
)

In [58]:
tuner5.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [59]:
tuner5.search((Wt5,Xt5), yt5, epochs=200,
           validation_split = val_percent,
           callbacks = [early_stop],
           verbose=2)

Trial 10 Complete [00h 46m 02s]
val_loss: 693.8296508789062

Best val_loss So Far: 668.3516540527344
Total elapsed time: 18h 51m 21s
INFO:tensorflow:Oracle triggered exit


In [60]:
tuner5.results_summary(1)

Results summary
Results in C:\RunsOak\10b-5
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 08 summary
Hyperparameters:
units: 66
filters: 512
kernel_size: 2
learning_rate: 0.001
Score: 668.3516540527344


### Best Model

In [61]:
best_hp5 = tuner5.get_best_hyperparameters()[0]
model5 = tuner5.hypermodel.build(best_hp5)

In [62]:
scaler_filename = "Best_HP/10b-5"
joblib.dump(best_hp5, scaler_filename) 

['Best_HP/10b-5']

In [63]:
scaler_filename = "Best_HP/10b-5"
best_hp5 = joblib.load(scaler_filename) 

model5 = tuner5.hypermodel.build(best_hp5)

In [64]:
model5.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 512 12800       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [65]:
model5.fit((Wt5,Xt5), yt5, validation_split = val_percent,
           epochs = 100, 
           verbose = 1)

Epoch 1/100
1231/1231 [==============================] - 152s 114ms/step - loss: 7949.7031 - mean_absolute_error: 59.3589 - root_mean_squared_error: 89.1611 - mean_absolute_percentage_error: 80.5669 - val_loss: 2050.3447 - val_mean_absolute_error: 31.4295 - val_root_mean_squared_error: 45.2807 - val_mean_absolute_percentage_error: 43.7618
Epoch 2/100
1231/1231 [==============================] - 136s 110ms/step - loss: 1540.4163 - mean_absolute_error: 27.8704 - root_mean_squared_error: 39.2481 - mean_absolute_percentage_error: 42.1035 - val_loss: 1167.8466 - val_mean_absolute_error: 24.7666 - val_root_mean_squared_error: 34.1738 - val_mean_absolute_percentage_error: 36.1319s: 1688.7596 - mean_ab - ETA: 40s - loss: 1650.2626 - mean_absolu
Epoch 3/100
1231/1231 [==============================] - 136s 111ms/step - loss: 1093.5210 - mean_absolute_error: 23.4951 - root_mean_squared_error: 33.0684 - mean_absolute_percentage_error: 36.3006 - val_loss: 862.9873 - val_mean_absolute_error: 21.057

1231/1231 [==============================] - 132s 107ms/step - loss: 56.3915 - mean_absolute_error: 5.5728 - root_mean_squared_error: 7.5094 - mean_absolute_percentage_error: 10.9913 - val_loss: 840.9172 - val_mean_absolute_error: 20.3829 - val_root_mean_squared_error: 28.9986 - val_mean_absolute_percentage_error: 26.9641_absolute_error: 5.5416 - root_mean_squared_error: 7.478 - ETA: 53s - loss: 55.7675 - mean_absolute_error: 5.5383 - - ETA: 32s - loss: 55.8568 - mean_absolute_error: 5.53 - ETA: 24s - loss: 55.9377  - ETA: 14
Epoch 44/100
1231/1231 [==============================] - 132s 107ms/step - loss: 53.9487 - mean_absolute_error: 5.4565 - root_mean_squared_error: 7.3450 - mean_absolute_percentage_error: 10.7928 - val_loss: 849.6854 - val_mean_absolute_error: 20.4936 - val_root_mean_squared_error: 29.1494 - val_mean_absolute_percentage_error: 26.9872
Epoch 45/100
1231/1231 [==============================] - 130s 106ms/step - loss: 51.0721 - mean_absolute_error: 5.3150 - root_mean

1231/1231 [==============================] - 132s 107ms/step - loss: 15.3226 - mean_absolute_error: 2.9128 - root_mean_squared_error: 3.9144 - mean_absolute_percentage_error: 6.0700 - val_loss: 856.3129 - val_mean_absolute_error: 20.7182 - val_root_mean_squared_error: 29.2628 - val_mean_absolute_percentage_error: 27.3761.4682 - mean_absolute_error: 2.8408 - root_mean_squared_error: 3. - ETA: 1:45 - loss: 15.3511 - m - ETA: 1:36 - loss: 14.9549 - mean_absolute_error: 2.8264 - root_mean_squared_error: 3.8671 - mean_absolute_perce - ETA: 1:34 - loss: 14.9023 - mean_absolute_error: 2.8313 - root_mean_squared_error: 3.8604 - mean_absolute_percentage_error:  - ETA: 1:33 - loss: 14.9708 - mean_absolute_error: 2.8361 - root_mean_squared_error: 3.8692 - mean_absolute_perc - ETA: 1:30 - loss: 14.9183 - mean_absolute_error: 2.8356 - root_mean_squared_error: 3.8624 - mean_absolute_percentage_error: 5.9 - ETA: 1:30 - loss: 14.8998 - mean_absolute_ - ETA: 45s - loss: 14.8746 - mean_absolute_error: 2

In [66]:
metrics5 = pd.DataFrame(model5.history.history)

In [67]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics5[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics5[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(400,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [68]:
model5.save("my_dir/10b-5")

INFO:tensorflow:Assets written to: my_dir/10b-5\assets


INFO:tensorflow:Assets written to: my_dir/10b-5\assets


In [69]:
model5 = keras.models.load_model("my_dir/10b-5") 

In [70]:
model5.predict((Wv5,Xv5))

array([[ 30.996784 ,  17.117014 ,  15.890254 ,  64.16951  ,  23.363234 ,
          9.781018 ],
       [ 29.156677 ,  12.4864855,  18.527386 ,  68.54899  ,  25.519182 ,
         10.760842 ],
       [ 30.466042 ,  15.949445 ,  11.495    ,  60.74011  ,  20.471987 ,
          9.905673 ],
       ...,
       [ 45.93892  ,  50.226963 ,  58.60002  , 145.95087  ,  53.74116  ,
         26.824274 ],
       [ 46.163532 ,  44.84726  ,  33.516754 ,  83.04904  ,  33.323006 ,
         26.332502 ],
       [ 41.85295  ,  33.47098  ,  33.912434 ,  87.96487  ,  34.42287  ,
         18.231922 ]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions5 = model5.predict((Wv5,Xv5))


MSE5 = mean_squared_error(yv5,test_predictions5)

MAE5 = mean_absolute_error(yv5,test_predictions5)

RMSE5 = np.sqrt(MSE5)

MAPE5 = mean_absolute_percentage_error(yv5,test_predictions5)*100

eval5 = [MSE5, MAE5, RMSE5, MAPE5]
print(eval5)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv5), np.asarray(test_predictions5))
GEH5 = count/GEH.size
print(GEH5)

# 15 mins ahead

In [73]:
def build_model_15(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt15.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt15.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_15w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_15 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_15w)
    
    
    
    # Deep Model
    # CNN 
    CNN_15 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_15 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_15)
    # LSTM          
    LSTM_15 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_15)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_15, LSTM_15], axis=1)

    # Attention
    Att_15 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_15 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_15.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_15

In [74]:
tuner15 = RandomSearch(
    build_model_15,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10b-15",
)

In [75]:
tuner15.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [76]:
tuner15.search((Wt15,Xt15), yt15, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop], 
            verbose=2)

Trial 10 Complete [01h 37m 17s]
val_loss: 746.0146789550781

Best val_loss So Far: 725.2720031738281
Total elapsed time: 10h 19m 17s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [77]:
tuner15.results_summary(1)

Results summary
Results in C:\RunsOak\10b-15
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 07 summary
Hyperparameters:
units: 77
filters: 320
kernel_size: 3
learning_rate: 0.001
Score: 725.2720031738281


### Best model

In [78]:
best_hp15 = tuner15.get_best_hyperparameters()[0]
model15 = tuner15.hypermodel.build(best_hp15)

In [79]:
scaler_filename = "Best_HP/10b-15"
joblib.dump(best_hp15, scaler_filename) 

['Best_HP/10b-15']

In [80]:
scaler_filename = "Best_HP/10b-15"
best_hp15 = joblib.load(scaler_filename) 

model15 = tuner15.hypermodel.build(best_hp15)

In [81]:
model15.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 320 17600       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [82]:
model15.fit((Wt15,Xt15), yt15, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
1231/1231 [==============================] - 105s 75ms/step - loss: 7364.4727 - mean_absolute_error: 57.0691 - root_mean_squared_error: 85.8165 - mean_absolute_percentage_error: 86.0703 - val_loss: 1780.9634 - val_mean_absolute_error: 30.6658 - val_root_mean_squared_error: 42.2015 - val_mean_absolute_percentage_error: 50.9635
Epoch 2/100
1231/1231 [==============================] - 92s 75ms/step - loss: 1608.2527 - mean_absolute_error: 28.8520 - root_mean_squared_error: 40.1030 - mean_absolute_percentage_error: 45.6764 - val_loss: 1207.7526 - val_mean_absolute_error: 25.6921 - val_root_mean_squared_error: 34.7527 - val_mean_absolute_percentage_error: 40.8795
Epoch 3/100
1231/1231 [==============================] - 92s 74ms/step - loss: 1298.0697 - mean_absolute_error: 25.2753 - root_mean_squared_error: 36.0287 - mean_absolute_percentage_error: 39.3671 - val_loss: 932.6059 - val_mean_absolute_error: 21.8125 - val_root_mean_squared_error: 30.5386 - val_mean_absolute_percentag

Epoch 43/100
1231/1231 [==============================] - 89s 72ms/step - loss: 940.9021 - mean_absolute_error: 20.5455 - root_mean_squared_error: 30.6741 - mean_absolute_percentage_error: 30.1633 - val_loss: 715.9720 - val_mean_absolute_error: 18.4321 - val_root_mean_squared_error: 26.7576 - val_mean_absolute_percentage_error: 26.0139
Epoch 44/100
1231/1231 [==============================] - 90s 74ms/step - loss: 941.2562 - mean_absolute_error: 20.5532 - root_mean_squared_error: 30.6799 - mean_absolute_percentage_error: 30.2122 - val_loss: 734.4565 - val_mean_absolute_error: 18.7033 - val_root_mean_squared_error: 27.1009 - val_mean_absolute_percentage_error: 24.5068
Epoch 45/100
1231/1231 [==============================] - 91s 74ms/step - loss: 938.5524 - mean_absolute_error: 20.5283 - root_mean_squared_error: 30.6358 - mean_absolute_percentage_error: 30.0938 - val_loss: 782.7988 - val_mean_absolute_error: 19.3677 - val_root_mean_squared_error: 27.9785 - val_mean_absolute_percentage_e

1231/1231 [==============================] - 177s 144ms/step - loss: 876.7466 - mean_absolute_error: 19.9276 - root_mean_squared_error: 29.6099 - mean_absolute_percentage_error: 29.0682 - val_loss: 745.3535 - val_mean_absolute_error: 18.7592 - val_root_mean_squared_error: 27.3012 - val_mean_absolute_percentage_error: 25.8565
Epoch 89/100
1231/1231 [==============================] - 150s 122ms/step - loss: 876.0521 - mean_absolute_error: 19.9358 - root_mean_squared_error: 29.5982 - mean_absolute_percentage_error: 29.1506 - val_loss: 736.5505 - val_mean_absolute_error: 18.6832 - val_root_mean_squared_error: 27.1395 - val_mean_absolute_percentage_error: 25.6932
Epoch 90/100
1231/1231 [==============================] - 170s 138ms/step - loss: 875.8683 - mean_absolute_error: 19.9274 - root_mean_squared_error: 29.5951 - mean_absolute_percentage_error: 29.1191 - val_loss: 769.4762 - val_mean_absolute_error: 19.0097 - val_root_mean_squared_error: 27.7394 - val_mean_absolute_percentage_error: 2

In [83]:
metrics15 = pd.DataFrame(model15.history.history)

In [84]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics15[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics15[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [85]:
model15.save("my_dir/10b-15")

INFO:tensorflow:Assets written to: my_dir/10b-15\assets


INFO:tensorflow:Assets written to: my_dir/10b-15\assets


In [86]:
model15 = keras.models.load_model("my_dir/10b-15")

In [87]:
model15.predict((Wv15,Xv15))

array([[ 29.745071,  18.38764 ,  16.281563,  66.88104 ,  32.504723,
          5.488085],
       [ 29.951706,  18.692717,  15.687355,  66.178314,  33.860817,
          6.326025],
       [ 28.53394 ,  16.752424,  16.232521,  63.110657,  31.676023,
          6.394144],
       ...,
       [ 45.861206,  47.8508  ,  72.72825 , 126.27006 ,  53.170338,
         23.103292],
       [ 41.77134 ,  41.762688,  66.18008 , 119.95129 ,  49.5751  ,
         21.250925],
       [ 39.86276 ,  38.848602,  63.853783, 121.30678 ,  51.824577,
         20.085047]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions15 = model15.predict((Wv15,Xv15))


MSE15 = mean_squared_error(yv15,test_predictions15)

MAE15 = mean_absolute_error(yv15,test_predictions15)

RMSE15 = np.sqrt(MSE15)

MAPE15 = mean_absolute_percentage_error(yv15,test_predictions15)*100

eval15 = [MSE15, MAE15, RMSE15, MAPE15]
print(eval15)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv15), np.asarray(test_predictions15))
GEH15 = count/GEH.size
print(GEH15)

# 30 mins ahead

In [90]:
def build_model_30(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt30.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt30.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_30w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_30 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_30w)
    
    
    # Deep Model
    # CNN 
    CNN_30 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_30 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_30)
    # LSTM          
    LSTM_30 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_30)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_30, LSTM_30], axis=1)

    # Attention
    Att_30 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_30 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_30.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_30

In [91]:
tuner30 = RandomSearch(
    build_model_30,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10b-30",
)

In [92]:
tuner30.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [93]:
tuner30.search((Wt30,Xt30), yt30, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [02h 07m 35s]
val_loss: 731.2030029296875

Best val_loss So Far: 728.2366333007812
Total elapsed time: 1d 10h 08m 29s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [94]:
tuner30.results_summary(1)

Results summary
Results in C:\RunsOak\10b-30
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 02 summary
Hyperparameters:
units: 68
filters: 448
kernel_size: 2
learning_rate: 0.01
Score: 728.2366333007812


### Best model

In [95]:
best_hp30 = tuner30.get_best_hyperparameters()[0]
model30 = tuner30.hypermodel.build(best_hp30)

In [96]:
scaler_filename = "Best_HP/10b-30"
joblib.dump(best_hp30, scaler_filename) 

['Best_HP/10b-30']

In [97]:
scaler_filename = "Best_HP/10b-30"
best_hp30 = joblib.load(scaler_filename) 

model30 = tuner30.hypermodel.build(best_hp30)

In [98]:
model30.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 448 11200       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [99]:
model30.fit((Wt30,Xt30), yt30, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
1231/1231 [==============================] - 412s 330ms/step - loss: 2103.4019 - mean_absolute_error: 29.1007 - root_mean_squared_error: 45.8629 - mean_absolute_percentage_error: 46.1656 - val_loss: 909.7741 - val_mean_absolute_error: 21.0282 - val_root_mean_squared_error: 30.1625 - val_mean_absolute_percentage_error: 30.8659
Epoch 2/100
1231/1231 [==============================] - 403s 327ms/step - loss: 1098.7954 - mean_absolute_error: 22.2752 - root_mean_squared_error: 33.1481 - mean_absolute_percentage_error: 33.7128 - val_loss: 793.1597 - val_mean_absolute_error: 19.8155 - val_root_mean_squared_error: 28.1631 - val_mean_absolute_percentage_error: 28.3713
Epoch 3/100
1231/1231 [==============================] - 403s 328ms/step - loss: 1064.4023 - mean_absolute_error: 21.9674 - root_mean_squared_error: 32.6252 - mean_absolute_percentage_error: 32.9084 - val_loss: 819.8615 - val_mean_absolute_error: 19.7703 - val_root_mean_squared_error: 28.6332 - val_mean_absolute_percen

1231/1231 [==============================] - 410s 333ms/step - loss: 929.9804 - mean_absolute_error: 20.5791 - root_mean_squared_error: 30.4956 - mean_absolute_percentage_error: 30.1202 - val_loss: 737.9985 - val_mean_absolute_error: 18.7564 - val_root_mean_squared_error: 27.1661 - val_mean_absolute_percentage_error: 26.5700
Epoch 50/100
1231/1231 [==============================] - 405s 329ms/step - loss: 931.3951 - mean_absolute_error: 20.5797 - root_mean_squared_error: 30.5188 - mean_absolute_percentage_error: 30.1735 - val_loss: 762.0256 - val_mean_absolute_error: 19.1337 - val_root_mean_squared_error: 27.6048 - val_mean_absolute_percentage_error: 29.2726
Epoch 51/100
1231/1231 [==============================] - 405s 329ms/step - loss: 924.9922 - mean_absolute_error: 20.4941 - root_mean_squared_error: 30.4137 - mean_absolute_percentage_error: 29.9588 - val_loss: 790.8867 - val_mean_absolute_error: 19.4133 - val_root_mean_squared_error: 28.1227 - val_mean_absolute_percentage_error: 2

Epoch 95/100
1231/1231 [==============================] - 111s 91ms/step - loss: 899.5347 - mean_absolute_error: 20.2909 - root_mean_squared_error: 29.9922 - mean_absolute_percentage_error: 29.5050 - val_loss: 798.4011 - val_mean_absolute_error: 19.5140 - val_root_mean_squared_error: 28.2560 - val_mean_absolute_percentage_error: 24.9027absolute_error: 20.3124 - root_mean_squared_error: 30.0254 - mean_absolute_percen
Epoch 96/100
1231/1231 [==============================] - 111s 90ms/step - loss: 896.9188 - mean_absolute_error: 20.2355 - root_mean_squared_error: 29.9486 - mean_absolute_percentage_error: 29.5542 - val_loss: 777.6954 - val_mean_absolute_error: 19.2552 - val_root_mean_squared_error: 27.8872 - val_mean_absolute_percentage_error: 26.4927
Epoch 97/100
1231/1231 [==============================] - 111s 90ms/step - loss: 893.5722 - mean_absolute_error: 20.1954 - root_mean_squared_error: 29.8927 - mean_absolute_percentage_error: 29.5218 - val_loss: 776.5025 - val_mean_absolute_er

In [100]:
metrics30 = pd.DataFrame(model30.history.history)

In [101]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics30[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics30[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [102]:
model30.save("my_dir/10b-30")

INFO:tensorflow:Assets written to: my_dir/10b-30\assets


INFO:tensorflow:Assets written to: my_dir/10b-30\assets


In [103]:
model30 = keras.models.load_model("my_dir/10b-30")

In [104]:
model30.predict((Wv30,Xv30))

array([[ 29.47062 ,  15.844233,  21.654182,  58.800125,  28.051598,
          5.032404],
       [ 29.693886,  16.61149 ,  20.415474,  57.353012,  28.644295,
          4.232416],
       [ 29.17591 ,  15.853594,  19.708809,  57.3239  ,  27.956581,
          3.427744],
       ...,
       [ 46.77053 ,  48.84504 ,  68.27515 , 112.54521 ,  45.824257,
         14.897516],
       [ 45.15996 ,  43.62641 ,  68.38453 , 115.6227  ,  44.14582 ,
         23.450586],
       [ 41.699085,  38.33522 ,  64.77076 , 119.36371 ,  50.48056 ,
         19.22252 ]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions30 = model30.predict((Wv30,Xv30))


MSE30 = mean_squared_error(yv30,test_predictions30)

MAE30 = mean_absolute_error(yv30,test_predictions30)

RMSE30 = np.sqrt(MSE30)

MAPE30 = mean_absolute_percentage_error(yv30,test_predictions30)*100

eval30 = [MSE30, MAE30, RMSE30, MAPE30]
print(eval30)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv30), np.asarray(test_predictions30))
GEH30 = count/GEH.size
print(GEH30)

# 60 mins ahead

In [107]:
def build_model_60(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt60.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt60.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_60w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_60 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_60w)
    
    
    # Deep Model
    # CNN 
    CNN_60 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_60 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_60)
    # LSTM          
    LSTM_60 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_60)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_60, LSTM_60], axis=1)

    # Attention
    Att_60 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_60 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_60.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_60

In [108]:
tuner60 = RandomSearch(
    build_model_60,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/RunsOak'),
    project_name="10b-60",
)

In [109]:
tuner60.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [110]:
tuner60.search((Wt60,Xt60), yt60, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [00h 19m 00s]
val_loss: 745.8909606933594

Best val_loss So Far: 732.4183349609375
Total elapsed time: 1d 16h 47m 56s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [111]:
tuner60.results_summary(1)

Results summary
Results in C:\RunsOak\10b-60
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 07 summary
Hyperparameters:
units: 48
filters: 384
kernel_size: 3
learning_rate: 0.01
Score: 732.4183349609375


### Best model

In [112]:
best_hp60 = tuner60.get_best_hyperparameters()[0]
model60 = tuner60.hypermodel.build(best_hp60)

In [113]:
scaler_filename = "Best_HP/10b-60"
joblib.dump(best_hp60, scaler_filename) 

['Best_HP/10b-60']

In [114]:
scaler_filename = "Best_HP/10b-60"
best_hp60 = joblib.load(scaler_filename) 

model60 = tuner60.hypermodel.build(best_hp60)

In [115]:
model60.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 6) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 6)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 384 21120       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 18)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [116]:
model60.fit((Wt60,Xt60), yt60, validation_split = val_percent,
            epochs = 100, 
            verbose = 1
            )

Epoch 1/100
1231/1231 [==============================] - 45s 33ms/step - loss: 2446.1450 - mean_absolute_error: 31.0834 - root_mean_squared_error: 49.4585 - mean_absolute_percentage_error: 48.3178 - val_loss: 926.9287 - val_mean_absolute_error: 21.4709 - val_root_mean_squared_error: 30.4455 - val_mean_absolute_percentage_error: 31.9039
Epoch 2/100
1231/1231 [==============================] - 38s 31ms/step - loss: 1098.5526 - mean_absolute_error: 22.2443 - root_mean_squared_error: 33.1444 - mean_absolute_percentage_error: 33.9268 - val_loss: 767.1633 - val_mean_absolute_error: 19.5921 - val_root_mean_squared_error: 27.6977 - val_mean_absolute_percentage_error: 34.6444
Epoch 3/100
1231/1231 [==============================] - 41s 34ms/step - loss: 1063.3062 - mean_absolute_error: 21.8966 - root_mean_squared_error: 32.6084 - mean_absolute_percentage_error: 32.8344 - val_loss: 802.3794 - val_mean_absolute_error: 19.8606 - val_root_mean_squared_error: 28.3263 - val_mean_absolute_percentage_e

1231/1231 [==============================] - 34s 27ms/step - loss: 926.1182 - mean_absolute_error: 20.5109 - root_mean_squared_error: 30.4322 - mean_absolute_percentage_error: 30.0944 - val_loss: 757.5894 - val_mean_absolute_error: 18.9413 - val_root_mean_squared_error: 27.5243 - val_mean_absolute_percentage_error: 24.7894
Epoch 50/100
1231/1231 [==============================] - 35s 29ms/step - loss: 926.1649 - mean_absolute_error: 20.5115 - root_mean_squared_error: 30.4330 - mean_absolute_percentage_error: 30.0133 - val_loss: 778.1261 - val_mean_absolute_error: 19.2871 - val_root_mean_squared_error: 27.8949 - val_mean_absolute_percentage_error: 27.4402
Epoch 51/100
1231/1231 [==============================] - 36s 29ms/step - loss: 922.9009 - mean_absolute_error: 20.4817 - root_mean_squared_error: 30.3793 - mean_absolute_percentage_error: 30.0352 - val_loss: 778.8002 - val_mean_absolute_error: 19.3450 - val_root_mean_squared_error: 27.9070 - val_mean_absolute_percentage_error: 26.7130

1231/1231 [==============================] - 48s 39ms/step - loss: 889.0920 - mean_absolute_error: 20.1611 - root_mean_squared_error: 29.8177 - mean_absolute_percentage_error: 29.4535 - val_loss: 808.3287 - val_mean_absolute_error: 19.7394 - val_root_mean_squared_error: 28.4311 - val_mean_absolute_percentage_error: 26.8876
Epoch 98/100
1231/1231 [==============================] - 46s 37ms/step - loss: 889.2863 - mean_absolute_error: 20.1490 - root_mean_squared_error: 29.8209 - mean_absolute_percentage_error: 29.3650 - val_loss: 785.4474 - val_mean_absolute_error: 19.2720 - val_root_mean_squared_error: 28.0258 - val_mean_absolute_percentage_error: 25.3225
Epoch 99/100
1231/1231 [==============================] - 46s 37ms/step - loss: 884.2193 - mean_absolute_error: 20.1197 - root_mean_squared_error: 29.7358 - mean_absolute_percentage_error: 29.3827 - val_loss: 773.7996 - val_mean_absolute_error: 19.1036 - val_root_mean_squared_error: 27.8173 - val_mean_absolute_percentage_error: 24.4568

In [117]:
metrics60 = pd.DataFrame(model60.history.history)

In [118]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics60[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics60[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [119]:
model60.save("my_dir/10b-60")

INFO:tensorflow:Assets written to: my_dir/10b-60\assets


INFO:tensorflow:Assets written to: my_dir/10b-60\assets


In [120]:
model60 = keras.models.load_model("my_dir/10b-60")

In [121]:
model60.predict((Wv60,Xv60))

array([[ 24.29293  ,   9.842206 ,  11.568355 ,  55.090828 ,  28.954958 ,
          5.781023 ],
       [ 25.346882 ,  11.718152 ,  11.0681715,  53.66239  ,  29.205431 ,
          5.7151737],
       [ 25.559696 ,  12.088765 ,   9.805598 ,  52.679497 ,  28.630014 ,
          5.50506  ],
       ...,
       [ 41.31996  ,  40.70625  ,  66.87429  , 117.21691  ,  51.308437 ,
         27.857714 ],
       [ 41.076206 ,  40.33038  ,  63.56648  , 116.12339  ,  52.203403 ,
         25.74917  ],
       [ 38.7423   ,  36.930073 ,  66.78251  , 120.95646  ,  54.61424  ,
         29.705645 ]], dtype=float32)

### Evaluation on inverse_transform

In [ ]:
test_predictions60 = model60.predict((Wv60,Xv60))


MSE60 = mean_squared_error(yv60,test_predictions60)

MAE60 = mean_absolute_error(yv60,test_predictions60)

RMSE60 = np.sqrt(MSE60)

MAPE60 = mean_absolute_percentage_error(yv60,test_predictions60)*100

eval60 = [MSE60, MAE60, RMSE60, MAPE60]
print(eval60)

In [ ]:
GEH, count = ga.GEH(np.asarray(yv60), np.asarray(test_predictions60))
GEH60 = count/GEH.size
print(GEH60)

# RESULTS

## General results

In [ ]:
l = ['MSE','MAE','RMSE','MAPE','Horizon']

In [ ]:
eval5.append(5)
eval15.append(15)
eval30.append(30)
eval60.append(60)

In [ ]:
df = pd.DataFrame(np.array([eval5,eval15,eval30,eval60]),columns=l)
df.set_index('Horizon')

## GEH

In [ ]:
n = [' 5 ',' 15 ',' 30 ',' 60 ']
df2 = pd.DataFrame(np.array([[GEH5, GEH15, GEH30, GEH60]]), columns=n)
print(df2)

## Save Loss

In [128]:
# metrics5.to_csv('Loss/8a-5.csv') 
# metrics15.to_csv('Loss/8a-15.csv') 
# metrics30.to_csv('Loss/8a-30.csv') 
# metrics60.to_csv('Loss/8a-60.csv') 